In this notebook we will guide you through our solution for the second homework of ADA.

# Part 1: Extracting the options from the dropdown menu

We start with the URL from the [homepage](http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247) where you can the select the different years/semesters etc..

In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys
import numpy as np


urlHome = 'http://isa.epfl.ch/imoniteur_ISAP/\
%21gedpublicreports.htm?ww_i_reportmodel=133685247'

We then use requests and BeautifulSoup to import the HTML data from the homepage.

In [14]:
home = requests.get(urlHome)
homeSoup = BeautifulSoup(home.content,'lxml')

The insepetion tool of our browser helped learn that we were to access a #document that is in the frame of one of the framesets of the HTML page, the document is sourced from another page of IS-Academia that we have to access. To access it, we scroll through the framesets and create a list of all the frames. 

In [15]:
iframe = []
for frame in homeSoup.frameset.find_all('frame'):
    iframe.append(frame)

urlOptions = iframe[1]['src']

print(urlOptions)

!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247


The url of the source page is found, we can use it to get the other HTML page. Again we use request and beautifulsoup 
to import the HTML data from this new page.

In [16]:
urlOptions = 'http://isa.epfl.ch/imoniteur_ISAP/' + urlOptions
options = requests.get(urlOptions)
soupOptions = BeautifulSoup(options.content,'lxml')





From this beautifulsoup object we can create a dictionary containing the name of each form parameters as key and the value of each parameters as value. We do this by scrolling to the option tag which contains this information.

In [17]:
#dictionary with the name and the value of each parameters
dic_name_value = {}

#Loop that passes trough all the option tag
for f in soupOptions.find_all('option'):
    
    #Filling of the dictionary
    dic = {f.string : f.attrs["value"]}
    dic_name_value.update(dic)


    
print(dic_name_value)

{'Ingénierie financière': '120623110', "Sciences et ingénierie de l'environnement": '942953', '2015-2016': '213638028', '2010-2011': '39486325', 'Projet Master printemps': '3781783', 'Mineur semestre 2': '2335676', 'Mise à niveau': '2063602308', 'Systèmes de communication': '946228', None: 'null', '2009-2010': '978195', '2016-2017': '355925344', 'Section FCUE': '1574548993', 'Bachelor semestre 5b': '2226768', '2014-2015': '213637922', 'Chimie et génie chimique': '246696', 'Semestre printemps': '2754553', 'Informatique': '249847', 'Bachelor semestre 6': '942175', 'Architecture': '942293', 'Bachelor semestre 4': '942163', 'Bachelor semestre 5': '942120', '2008-2009': '978187', 'Master semestre 2': '942192', 'Semestre automne': '953159', 'Cours de mathématiques spéciales': '943282', 'Génie civil': '942623', 'Bachelor semestre 3': '942155', 'Stage printemps master': '2227132', 'Master semestre 1': '2230106', '2011-2012': '123455150', 'Projet Master automne': '249127', 'Stage printemps 4ème

We separate this dictionary into smaller dictionaries specific to each form parameters. Which means one for the section, one for the academic period, etc. We do this specifically for the bachelor students, in order to complete the first task.

In [18]:
#From the dictionnary we created before with the name and the value, we extract the parameters that we want. 
#We create separated dictionary for every form parameters.

#First parameter: Informatique
parameter_section = {'Informatique' : (dic_name_value['Informatique'])}
print(parameter_section)


#Seconde parameter: Semester

#Initialization of the dictionary
parameter_semester = {}



#Loop that will pass trough the whole dictionary and select the ones that start with Semestre
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith('Semestre d'):
            parameter_semester[key]=value
print(parameter_semester)


#Third parameter: years
parameter_years = {}

for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith('2'):
            parameter_years[key]=value
            
print(parameter_years)


#Fourth parameter: Pedagogical period

test = ["Ba"]   #test = ["Ba", "Ma","Min","Mis","Pro","Sem","Sta"]
parameter_period = {};
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith(tuple(test)):
            parameter_period[key]=value
print(parameter_period)



{'Informatique': '249847'}
{"Semestre d'automne": '2936286', 'Semestre de printemps': '2936295'}
{'2011-2012': '123455150', '2014-2015': '213637922', '2009-2010': '978195', '2010-2011': '39486325', '2016-2017': '355925344', '2013-2014': '213637754', '2012-2013': '123456101', '2007-2008': '978181', '2015-2016': '213638028', '2008-2009': '978187'}
{'Bachelor semestre 4': '942163', 'Bachelor semestre 5b': '2226768', 'Bachelor semestre 3': '942155', 'Bachelor semestre 6b': '2226785', 'Bachelor semestre 1': '249108', 'Bachelor semestre 6': '942175', 'Bachelor semestre 5': '942120', 'Bachelor semestre 2': '249114'}


From theses dictionaries, we load the data directly from the IS-academia servers using pd.read_html. We use the standard url and we change the form parameters values for each values in each dictionaries using the %s method.

In [19]:
# Now we want to request the is-academia servers for the specific data we need, using the dictionaries that we created
#before. We will use for loop passing trough every dictionaries in order to compose the multiple requests needed.

data_all = pd.DataFrame()

#Multiple loop that permits to use each values in each dictionaries
for value_section in parameter_section.values():
    for value_years in parameter_years.values():
        for value_period in parameter_period.values():
            for value_semester in parameter_semester.values():
                
                #We use a try to prevent the problem when there is a empty list.
                try:
                    #The url is changed for every values in each loops
                    student_data = pd.read_html('http://isa.epfl.ch/imoniteur_ISAP//!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=%s&ww_x_PERIODE_ACAD=%s&ww_x_PERIODE_PEDAGO=%s&ww_x_HIVERETE=%s' % (value_section, value_years, value_period, value_semester))
                    
                    #The data is reshaped to be more suite for subsequent analysis
                    student_data.columns = student_data.loc[[1]].values.reshape(12,)
                    student_data.drop(student_data.index[[0,1]],inplace=True)
                    student_data = student_data[['Civilité','Nom Prénom',]]
                except:
                    pass
                
                #Each block of data is concatenated to have a unique dataframe
                data_all = pd.concat([data_all, student_data[0]], axis = 0)
                
data_all



,0,1,2,3,4,5,6,7,8,9,10,11
0,"Informatique, 2008-2009, Bachelor semestre 2 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Monsieur,Ahluwalia Samit,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,184228,NaN
3,Monsieur,Amiet Nils,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179383,NaN
4,Monsieur,Arnfred Jonas,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,184772,NaN
5,Monsieur,Bougacha Malik,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,186264,NaN
6,Monsieur,Bourgeois Jérémy,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,185949,NaN
7,Monsieur,Bourqui Marc,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,184264,NaN
8,Monsieur,Bruchez Lionel,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,186825,NaN
9,Monsieur,Cambazard Thomas,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179864,NaN
